In [13]:
import cv2
import numpy as np
import serial
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [14]:
cam = 2
fps = 240

model_version = "my_model_14.0.keras"
Im_Width = 128
Im_Height = 128

In [15]:
def setup_serial_connection(port='COM4', baud_rate=9600):
    arduino = serial.Serial(port, baud_rate, timeout=1)
    print("Arduino connected. Waiting for initialization...")
    arduino.flush()  # Clear any remaining data
    return arduino

In [16]:
def send_to_arduino(arduino, value):
    arduino.write(f"{value}\n".encode())
    print(f"Sent: {value}")

In [17]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [18]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [19]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [20]:
def select_best_frame(frame_buffer):

    best_frame = None
    highest_brightness = 150
    highest_edge_richness = 100000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, 220, 0, 200, 200)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > 125:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
    
        elif brightness > 50 and brightness < 100:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame

    return best_frame  # Returns None if no frames meet the threshold


In [21]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [22]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        
        return "Non-Defective", prediction
        
        
    else:
        send_to_arduino(arduino, 0)
        return "Defective", prediction
    


In [23]:
def capture(cam, fps, model_path, target_size=(Im_Width, Im_Height)):

    cap = cv2.VideoCapture(cam)  # Open webcam
    # Set desired frame rate
    desired_fps = fps
    cap.set(cv2.CAP_PROP_FPS, desired_fps)

    frame_buffer = []
    max_frames = 5  # Number of frames to evaluate

    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            if best_frame is not None:
                feedback, confidence = predict_frame(best_frame, model, target_size)
                print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

                cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(best_frame, f"Confidence: {confidence:.2f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
                # Display the best frame
                cv2.imshow("Best Frame", best_frame)
                if feedback == "defective" :
                    send_to_arduino(arduino, 0)
                else:
                    send_to_arduino(arduino, 1)
            
            else:
                send_to_arduino(arduino, 1)
                '''
                image_path = "no-bottle.bmp"
                image = cv2.imread(image_path)
                # Display the image in a window
                cv2.putText(best_frame, f"No Bottle!", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Pre-Saved Image", image)'''

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

In [24]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    arduino = setup_serial_connection('COM4')  # Connect once
    capture(cam, fps, model_path, target_size=(Im_Width, Im_Height))

Arduino connected. Waiting for initialization...
Loading model...
Model loaded successfully.
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sent: 1
Sen